## Load Tile Placements Table

In [11]:
import sqlite3
conn = sqlite3.connect('../dataset/tile_placements.db')
c = conn.cursor()

In [ ]:
conn.close()

---

## Create Tile Placements Table

In [2]:
c.execute('''CREATE TABLE tile_placements 
            (timestamp INTEGER, user_hash TEXT, user_name TEXT, x_coord INTEGER, y_coord INTEGER, color INTEGER)''')
conn.commit()

---

## Insert CSV File to Tile Placements Table 

In [5]:
def insert_tile_placement(timestamp, user_hash, user_name, x_coord, y_coord, color):
    c.execute('''INSERT INTO tile_placements (timestamp, user_hash, user_name, x_coord, y_coord, color)
                 VALUES (?, ?, ?, ?, ?, ?);''', (timestamp, user_hash, user_name, x_coord, y_coord, color))

In [6]:
read_data = open("../../ShortcutRedditDir/RawData/place/tile_placements_with_usernames.csv").readlines()[1:]
for line in read_data:
    line = line.split(',')
    insert_tile_placement(int(line[0])/1000, line[1], line[2],  int(line[3]), int(line[4]), int(line[5]))
conn.commit()

---

## Empty Table

In [ ]:
c.execute("DELETE FROM tile_placements;")
conn.commit()

---

##  Show Size of Table 

In [12]:
c.execute('SELECT COUNT(*) FROM tile_placements;')
print (c.fetchone())

(16559897,)


---

##  Show Contents of Table 

In [ ]:
c.execute('SELECT * FROM tile_placements;')
print (c.fetchone())

---

##  Generate Index for Table 

In [8]:
c.execute('CREATE INDEX idx_placement_username ON tile_placements (user_name)')

In [9]:
c.execute('CREATE INDEX idx_placement_coord ON tile_placements (x_coord, y_coord)')

In [13]:
c.execute('CREATE INDEX idx_placement_timestamp ON tile_placements (x_coord, y_coord, timestamp)')

---

##  Search Contents of Table 

In [10]:
%%time
c.execute("SELECT * FROM tile_placements WHERE user_name = 'bolush_it';")
print (c.fetchall())

[(1491167838, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 326, 258, 0), (1491136027, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 66, 993, 0), (1491010730, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 631, 113, 0), (1491097193, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 851, 30, 4), (1491055635, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 308, 259, 5), (1491137819, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 904, 702, 13), (1491054873, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 317, 255, 0), (1491079626, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 311, 261, 0), (1490985199, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 34, 624, 3), (1491057853, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 308, 259, 5), (1490980932, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 500, 497, 3), (1490986755, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 22, 633, 3), (1491164149, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 910, 703, 8), (1491059470, 'c2Md9m4QckH8M+lhNh9btgoiFDk=', 'bolush_it', 296, 270

---

##  Get Unique UserNames in Table 

In [ ]:
%%time
c.execute("SELECT DISTINCT user_name FROM tile_placements;")
print (c.fetchall())